In [1]:
%pip install nltk pandas transformers
%pip install SpeechRecognition pydub
%pip install -U jupyter ipywidgets tqdm
%pip install --upgrade setuptools
%pip install tf-keras
%pip install torch torchvision torchaudio
%pip install tensorflow
%pip install tensorflow --upgrade



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Import required libraries
import speech_recognition as sr
import nltk
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from pydub import AudioSegment
from transformers import pipeline, AutoTokenizer
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/hoobot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hoobot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Speech to Text

In [4]:
# Convert MP3 to WAV for easier processing by SpeechRecognition
def convert_audio(input_file):
    sound = AudioSegment.from_mp3(input_file)
    output_file = "converted_file.wav"
    sound.export(output_file, format="wav")
    return output_file

In [5]:
 # Transcribe audio to text
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Google Speech Recognition could not understand audio"
        except sr.RequestError as e:
            return "Could not request results from Google Speech Recognition service; {0}".format(e)



In [6]:
# Split video to 30 second chunks for faster processing and less chance of error
def convert_and_chunk_audio(input_file, output_folder="audio_chunks", chunk_length_ms=30000):
    # Create the output folder if it doesn't already exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    sound = AudioSegment.from_mp3(input_file)
    chunks = []

    for i in range(0, len(sound), chunk_length_ms):
        chunk = sound[i:i+chunk_length_ms]
        chunk_name = f"chunk{i//chunk_length_ms}.wav"
        chunk_path = os.path.join(output_folder, chunk_name)  # Save chunks to the specified folder
        chunk.export(chunk_path, format="wav", parameters=["-ar", "16000"])
        chunks.append(chunk_path)

    return chunks

In [7]:
def parallel_transcribe_audio(chunks):
    transcriptions = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_chunk = {executor.submit(transcribe_audio, chunk): chunk for chunk in chunks}
        for future in as_completed(future_to_chunk):
            chunk = future_to_chunk[future]
            try:
                transcription = future.result()
                transcriptions.append(transcription)
            except Exception as exc:
                print(f'Chunk {chunk} generated an exception: {exc}')
    return " ".join(transcriptions)

In [8]:
def process_audio_file(input_file):
    chunks = convert_and_chunk_audio(input_file)
    full_transcription = parallel_transcribe_audio(chunks)
    print("Full Transcription:", full_transcription)

    with open("transcribed_text.txt", "w") as text_file:
        text_file.write(full_transcription)

## Driver Code

In [9]:
# Replace 'Interview_Audio.mp3' with the exact audio file
# process_audio_file("Exit Interview Questions from We.mp3")
process_audio_file("Why I left my job at Google (as a software engineer).mp3")

Full Transcription: and you know it was it was great not before I got into Google I would be one of those people who would just go like Costco and just get as many samples as I could and that would be my lunch I would love food samples and I might go Trader Joe's and grab their free coffee and I might go to chocolate shops and just get free samples everywhere and you know I was just hungry and starving basically everywhere I went and after Google that kind of changed me a little bit basically I was able to get as many free samples as I wanted that Google because hi and welcome back to another episode of the tech lead my name is the tech lead and I will be the tech lead today I wanted to talk about why I left my six-figure job at Google because for a lot of people they imagine that once they get into Google it will be happily ever after from there on out like it's a great company to work for it's kind of the lifelong dream for a lot of people and once they get in they imagine they'll ju

## Sentiment Analysis

In [18]:
def analyze_sentiment(text):
    # Load the tokenizer and sentiment-analysis pipeline
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    classifier = pipeline('sentiment-analysis', model="bert-base-uncased")

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    
    sentiment_results = []
    for sentence in sentences:
        # Check if the length of the tokens does not exceed the maximum size
        tokens = tokenizer.tokenize(sentence)
        if len(tokens) > 510:
            tokens = tokens[:510]  # truncate tokens if they are too long
        # Convert tokens back to string
        sentence_text = tokenizer.convert_tokens_to_string(tokens)
        # Analyze sentiment of the sentence
        result = classifier(sentence_text)
        sentiment_results.extend(result)

    return sentiment_results

In [19]:
# Analyze on sample text for better accuracy
sample_text_file = "sample.txt"
with open(sample_text_file, "r") as file:
    sample_text = file.read()

# Analyze sentiment per sentence
sentiment_results = analyze_sentiment(sample_text)

# Process and print results
for result in sentiment_results:
    print(result)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'label': 'LABEL_0', 'score': 0.6816516518592834}
{'label': 'LABEL_0', 'score': 0.6877008676528931}
{'label': 'LABEL_0', 'score': 0.6985224485397339}
{'label': 'LABEL_0', 'score': 0.699279248714447}
{'label': 'LABEL_0', 'score': 0.700007975101471}
{'label': 'LABEL_0', 'score': 0.6885144710540771}
{'label': 'LABEL_0', 'score': 0.7042182087898254}
{'label': 'LABEL_0', 'score': 0.6901777982711792}
{'label': 'LABEL_0', 'score': 0.6826658844947815}
{'label': 'LABEL_0', 'score': 0.6771817803382874}
{'label': 'LABEL_0', 'score': 0.6870958209037781}
{'label': 'LABEL_0', 'score': 0.673376202583313}
{'label': 'LABEL_0', 'score': 0.6729334592819214}
{'label': 'LABEL_0', 'score': 0.677141547203064}
{'label': 'LABEL_0', 'score': 0.7016158699989319}
{'label': 'LABEL_0', 'score': 0.6890334486961365}
{'label': 'LABEL_0', 'score': 0.6983551383018494}
{'label': 'LABEL_0', 'score': 0.6702356934547424}
{'label': 'LABEL_0', 'score': 0.6795105934143066}
{'label': 'LABEL_0', 'score': 0.6782650351524353}
{'la